In [2]:
import csv 
import json 
import pandas as pd
pd.set_option("display.max_columns", None)

In [37]:
df = pd.read_csv('hr_raw_data.csv', index_col=0)
df.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1


In [ ]:
#Cambio inicial de headers, pasando todos los títulos de columnas a minúscula:
nuevas_columnas = {columna: columna.lower() for columna in df.columns}
df.rename(columns=nuevas_columnas, inplace= True)

#Columna Age - REVISAR. LAS NOTAS SON ALGO CAÓTICAS
#Attrition - ok. No changes
#Businesstravel - ok. No changes
#Dailyrate - se redondea a 2
df['dailyrate']=df['dailyrate'].round(2)
#Department - está bien. tiene nulos. como sugerencia estaría bien rellenar departamentos en caso de que haya más que no se contemplan.
#DistanceFromHome - numeros negativos. Cambiar float
df['distancefromhome'] = df['distancefromhome'].abs() #método para convertir números negativos en positivos abs. tipo int: correcto
#Education - todo bien
#EducationField - todo bien
#Employeecount - drop porque no tiene demasiado sentido esta columna
df.drop('employeecount', axis= 1, inplace= True)
#Employeenumber - PARA REVISAR TODAVÍA.
#EnvironmentSatisfaction - solo nos quedamos con el primer número y hacemos que todo sea integer
df["environmentsatisfaction"]= df["environmentsatisfaction"].astype(str).str[0].astype(int)
#Gender - transformar el 0 y 1 por M y F
df['gender'] = df['gender'].replace(0, 'M')
df['gender'] = df['gender'].replace(1, 'F')
#HourlyRate - se ha decidido hacer drop
df.drop('hourlyrate', axis= 1, inplace= True)
#JobInvolvement - todo bien
#JobLevel - todo bien
#JobRole - escrito en leet. Cambiamos todo a minúsculas.
df['jobrole'] = df['jobrole'].str.lower()
#JobSatisfaction - todo bien
#MaritalStatus - corregidos errores ortográficos y modificado todo a minúsculas
df['maritalstatus'] = df['maritalstatus'].str.lower()
df['maritalstatus'] = df['maritalstatus'].replace("marreid", "married")
#MonthlyIncome - PARA REVISAR. SE HICIERON COSAS AYER PERO COMO HABÍA LIADAS LA DEJO PENDIENTE JUST IN CASE
#MonthlyRate - quitamos el símbolo de dolar, cambiamos comas por puntos, pasamos a float
df['monthlyrate'] = df['monthlyrate'].str.replace('$', '', regex=False)
df['monthlyrate'] = df['monthlyrate'].str.replace(',', '.', regex=False)
df['monthlyrate'] = df['monthlyrate'].astype(float)
#NUMCOMPANIESWORKED - todo bien
#OverTime - PARA REVISAR. creo que no se hizo nada.
#PercentSalaryHike - todo bien
#PerformanceRating - todo bien? los valores solo son 3,0, 4,0, nan. type object. Tal vez deberíamos de pasarlo a INT? sino queda como object
#RelationshipSatisfaction - todo bien
#StandardHours - object, full-time y part-time, Nan de esta columna son full time, hay que corregir, #va del 1 al 4, mantener los nulos, intentar profundizar con el job role o job department
#StockOptionLevel - todo bien. nivel de opciones sobre acciones. Va de 0 a 3
#TrainingTimesLastYear - todo bien . int de 0 a 6
#WORKLIFEBALANCE - todo bien. object de 1,0 a 4,0 y nan.
#YearsAtCompany - todo bien.
#YearsInCurrentRole - muchos no contestan. por lo demás se ve bien. está como object. tal vez se pudiese cambiar a float.
#YearsSinceLastPromotion - todo bien. escala del 0 al 15.
#YEARSWITHCURRMANAGER - todo bien. min value 0, max value en la lista 17. int.
#DateBirth - todo bien.
#Salary - PARA REVISAR. se miró hoy. unsure cómo quedó la cosa.
#RoleDepartament - tenemos rol y tenemos departamento. yo me cargaría esta columna. No le veo sentido
df.drop('roledepartament', axis= 1, inplace= True)
#NUMBERCHILDREN - solo tiene nan así que la droppeamos. recomendar para conciliación/worklifebalance.
df.drop('numberchildren', axis= 1, inplace= True)
#RemoteWork - cambiamos las respuestas para que sean Yes o No , sustituyendo 1 y True por Yes, y 0 y False por No
df['remotework'].replace("1", 'Yes', inplace= True)
df['remotework'].replace("0", 'No', inplace= True)
df['remotework'].replace("False", 'No', inplace= True)
df['remotework'].replace("True", 'Yes', inplace= True)




In [ ]:
#funciones

# Poner todos los nombres de columnas en minúsculas
def poner_headers_minusculas(df):
    nuevas_columnas = {columna: columna.lower() for columna in df.columns}
    df.rename(columns=nuevas_columnas, inplace=True)

# Redondear los valores de una columna a 2 decimales
def redondear_columna(df, nombre_columna):
    df[nombre_columna] = df[nombre_columna].round(2)

# Cambiar valores negativos por positivos en una columna
def hacer_valores_positivos(df, nombre_columna):
    df[nombre_columna] = df[nombre_columna].abs()

# Eliminar columnas que no se necesitan
def eliminar_columnas(df, lista_columnas):
    df.drop(lista_columnas, axis=1, inplace=True)

# Dejar solo el primer número de una columna y convertirlo en entero
def primer_numero_a_entero(df, nombre_columna):
    df[nombre_columna] = df[nombre_columna].astype(str).str[0].astype(int)

# Reemplazar valores de una columna según un diccionario
def reemplazar_valores(df, nombre_columna, reemplazos):
    df[nombre_columna] = df[nombre_columna].replace(reemplazos)

# Convertir los textos de una columna a minúsculas
def poner_columna_minusculas(df, nombre_columna):
    df[nombre_columna] = df[nombre_columna].str.lower()

# Limpiar columna de dinero: quitar $ y cambiar , por . luego convertir a número decimal
def limpiar_columna_dinero(df, nombre_columna):
    df[nombre_columna] = df[nombre_columna].str.replace('$', '', regex=False)
    df[nombre_columna] = df[nombre_columna].str.replace(',', '.', regex=False)
    df[nombre_columna] = df[nombre_columna].astype(float)

# Convertir una columna a tipo decimal (float)
def convertir_a_decimal(df, nombre_columna):
    df[nombre_columna] = pd.to_numeric(df[nombre_columna], errors='coerce')

In [ ]:
#ejemplos de uso
poner_headers_minusculas(df)

redondear_columna(df, 'dailyrate')
hacer_valores_positivos(df, 'distancefromhome')
eliminar_columnas(df, ['employeecount', 'hourlyrate', 'roledepartament', 'numberchildren'])
primer_numero_a_entero(df, 'environmentsatisfaction')

reemplazar_valores(df, 'gender', {0: 'M', 1: 'F'})
poner_columna_minusculas(df, 'jobrole')
poner_columna_minusculas(df, 'maritalstatus')
reemplazar_valores(df, 'maritalstatus', {'marreid': 'married'})

limpiar_columna_dinero(df, 'monthlyrate')

reemplazar_valores(df, 'remotework', {
    "1": 'Yes', "0": 'No', "False": 'No', "True": 'Yes'
})